In [6]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')
import datetime
import lightgbm as lgb
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split, KFold
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler
stop = set(stopwords.words('english'))
import os

from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
import json
import ast
import eli5
from functools import reduce
import warnings
from sklearn.metrics import mean_squared_error
from scipy import stats
from math import sqrt
from lightgbm import plot_tree
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from hyperopt import Trials
from hyperopt import fmin
from hyperopt import STATUS_OK
from hyperopt.pyll.stochastic import sample
import time
import gc
warnings.filterwarnings('ignore')
#import shap

In [55]:
train = pd.read_csv('dataset-0510/train.csv')
test  = pd.read_csv('dataset-0510/test.csv')
data = pd.concat([train, test], axis=0)
#drop outlier
train.drop(train[(train['land_area'] > 1500) | (train['building_area'] >1000)].index, inplace= True)

In [56]:
def Feature_engineer(df):
    #floor
    '''
    df.loc[df['txn_floor'].isna(), 'department'] = 1 
    df.loc[df['txn_floor'].notna(), 'department'] = 0
    df.loc[df['txn_floor'].isna(), 'txn_floor'] = df.loc[df['txn_floor'].isna(), 'total_floor'] 
    df['avg_height_floor'] = df['txn_floor'] / df['total_floor'] 
    df['avg_height_floor'].fillna(0, inplace = True)
    '''
    df['avg_height_floor'] = df['txn_floor'] / df['total_floor'] 
    #location
    df['location_2'] = df.apply(lambda x : int(str(x['city']) + str(x['town'])), axis=1)
    df['city'] = df['city'].astype('category')
    df['location_2'] = df['location_2'].astype('category')
    
    #date
    df['day_between_txn_complete'] = df['txn_dt'] - df['building_complete_dt']
    df['year_between_txn_complete'] = round(df['day_between_txn_complete'] / 365)
    
    #area
    df['land/bulid_area'] = df['land_area'] / df ['building_area']
    df['house_area'] = df['building_area'] - df['parking_area'] 
    df['house_rate'] = df['building_area'] / df['house_area']
    df['area_rate'] = df['land_area'] / df['town_area']
    
    
    #parking
    df['miss_parking_area'] = 0
    df['miss_parking_price'] = 0
    df.loc[df['parking_area'].isna(), 'miss_parking_area'] = 1
    df.loc[df['parking_price'].isna(), 'miss_parking_price'] = 1
    #df['parking_price_every_area'] = df['parking_price'] / df['parking_area']
    #df['parking_way'] = df['parking_way'].astype('category')
    df = pd.concat([df, pd.get_dummies(df['parking_way'], prefix='parking_way')], axis=1)

    #building   #building_type = 4 is house
    #df['building_type'] = df['building_type'].astype('category')
    #df['building_material'] = df['building_material'].astype('category')
    #df['building_use'] = df['building_use'].astype('category')
    
    #df['building_material'] = df['building_material'].apply(lambda x : 2 if x not in [1,8] else x)
    #df['building_use'] = df['building_use'].apply(lambda x : 2 if x not in [1,2,5] else x)
    
    df = pd.concat([df, pd.get_dummies(df['building_material'], prefix='building_material')], axis=1)
    df = pd.concat([df, pd.get_dummies(df['building_use'], prefix='building_use')], axis=1)  
    
    #10 50 100 250 500 1000 5000 10000
    df['diff_500_50'] = df['N_500'] - df['N_50']
    df['diff_1000_500'] = df['N_1000'] - df['N_500']
    df['diff_5000_1000'] = df['N_5000'] - df['N_1000']
    df['diff_10000_5000'] = df['N_10000'] - df['N_5000']
    df['rate_500_50'] = df['N_500'] / df['N_50']
    df['rate_1000_500'] = df['N_1000'] / df['N_500']
    df['rate_5000_1000'] = df['N_5000'] / df['N_1000']
    df['rate_10000_5000'] = df['N_10000'] / df['N_5000']
    
    #MIN
    MIN_cols = [i for i in df.columns if '_MIN' in i]
    df[MIN_cols].apply(lambda x : x.mean(), axis =1 )
    df[MIN_cols].apply(lambda x : x.median(), axis =1 )
    df[MIN_cols].apply(lambda x : x.std(), axis =1 )
    df[MIN_cols].apply(lambda x : x.skew(), axis =1 )
    
    rank_df = df[MIN_cols].rank(axis =1)
    rank_df = rank_df.add_prefix('Rank_')
    df = pd.concat([df, rank_df], axis=1)
    
    category_cols  = ['city', 'building_material', 'building_use', 'parking_way']
    '''
    numerical_cols = ['building_area', 'land_area', 'day_between_txn_complete', 'txn_dt', 'building_complete_dt']
    statistics = ['mean', 'median', 'max', 'min']
    for category in category_cols:
        for numerical in numerical_cols:
            for stat in statistics:
                df[numerical + '_' + stat + '_gb_' + category] = df.groupby([category])[numerical].transform(stat)
                df[numerical + '_diff_' + stat + '_gb_' + category] = df[numerical] - df[numerical + '_' + stat + '_gb_' + category]
                df.drop([numerical + '_' + stat + '_gb_' + category], inplace = True, axis = 1)
    
    #interection
    
    inter_cols = ['parking_way', 'building_use', 'building_material']
    for i in range(3):
        for j in range(3):
            if j > i:
                df['inter_btw_' + inter_cols[i] + '_' + inter_cols[j]] = df.apply(lambda x : str(x[inter_cols[i]]) + str(x[inter_cols[j]]), axis=1)
                df['inter_btw_' + inter_cols[i] + '_' + inter_cols[j]] = df['inter_btw_' + inter_cols[i] + '_' + inter_cols[j]].astype('category')
    '''
    category_cols  = ['building_material',
                      'building_type',
                      'building_use',
                      'parking_way']
    
    numerical_cols = ['building_area', 'land_area', 'day_between_txn_complete']
    statistics = ['mean', 'median', 'max', 'min']
    for category in category_cols:
        for numerical in numerical_cols:
            for stat in statistics:
                df[numerical + '_' + stat + '_gb_' + category] = df.groupby([category])[numerical].transform(stat)
                df[numerical + '_diff_' + stat + '_gb_' + category] = df[numerical] - df[numerical + '_' + stat + '_gb_' + category]
                df.drop([numerical + '_' + stat + '_gb_' + category], inplace = True, axis = 1)
    
    
    df.drop(['building_use', 'building_material', 'parking_area', 'parking_price', 'parking_way'], axis = 1, inplace = True)
    df.drop(['village'], axis = 1, inplace = True)
    df.drop([i for i in df.columns if 'index' in i and np.sum(df[i]) == 70000], axis = 1, inplace = True)
    return df

In [43]:
def c_hit(preds, train_data):
    trues  = train_data.get_label()
    trues = np.expm1(trues)
    preds = np.expm1(preds)
    scores = ((np.absolute(preds - trues) / trues) <= 0.1)
    hit_error = np.sum(scores) / train_data.num_data()
    return 'Hit_Score', round(hit_error * 10000), True

def Submission(Ids, preds):
    file_name = datetime.datetime.today().strftime('%m-%d-%H-%M')
    submission = pd.DataFrame({'building_id' : Ids, 'total_price' : preds})
    if not os.path.isdir('Submission'):
        os.makedirs('Submission')
    submission.to_csv('Submission/' + file_name + '.csv', index= False)

In [44]:
def display_importances(feature_importance_df):
    cols = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()

In [60]:
def lgb_model(split_num, train, test,cat_cols, stratified = False):
    

    if stratified:
        kf = StratifiedKFold(n_splits = split_num, random_state = 42, shuffle = True)
    else :
        kf = KFold(n_splits = split_num, random_state=42, shuffle=True)
    train['total_price_log'] = np.log1p(train['total_price'])
    feature_importance_df = pd.DataFrame()
    oof = np.zeros(len(train))
    predictions = np.zeros(len(test))
    

    param ={
        'n_estimators': 1, 'max_depth' : 8, 'num_leaves' :128,         
        'objective': 'regression',   'metric':'rmse',   
        'learning_rate': 0.01,      'boosting': 'gbdt',     'min_data_in_leaf': 10,
        'feature_fraction': 0.9,    'bagging_freq':1,       'bagging_fraction': 0.9,     'importance_type': 'gain',
        'lambda_l1': 0.1,  'subsample': .8,   'colsample_bytree': .8, 'device' : 'gpu', 'num_trhead' : 10
    }


    features = [i for i in train.columns if i not in ['building_id','total_price_log', 'building_type', 'total_price']] 
    print(len(features))       
    for fold_, (trn_idx, val_idx) in enumerate(kf.split(train[features].values,train['total_price_log'].values)):
        trn_data = lgb.Dataset(train.iloc[trn_idx][features], label= train['total_price_log'].iloc[trn_idx])
        val_data = lgb.Dataset(train.iloc[val_idx][features], label= train['total_price_log'].iloc[val_idx])
        
        
        clf = lgb.train(params= param, train_set= trn_data, valid_sets= [trn_data, val_data], verbose_eval=3000, early_stopping_rounds= 5000, categorical_feature=cat_cols, feval=lambda preds, train_data: [c_hit(preds, train_data)],)
        oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration = clf.best_iteration)
        predictions += clf.predict(test[features], num_iteration = clf.best_iteration) / kf.n_splits
        
        y   = np.expm1(train['total_price_log'].iloc[val_idx]) 
        yhat = np.expm1(oof[val_idx])
        Hit_score = np.sum([1 for i in np.abs((y - yhat) / y)  if i <= 0.1 ])
        print('fold {} hit_score : {}'.format(fold_ + 1, round(Hit_score, 4) /len(train.iloc[val_idx]) * 10000))
        print('-'*30)
        #fold_importance_df = pd.DataFrame()
        #fold_importance_df['feature']    = features
        #fold_importance_df['importance'] = np.log1p(clf.feature_importance(importance_type='gain', iteration=clf.best_iteration))
        #fold_importance_df['fold']       = fold_ + 1
        #feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    print('CV scrore : {}'.format(sqrt(mean_squared_error(train['total_price_log'], oof))))
    print('-'*30)
    y = np.expm1(train['total_price_log']) 
    yhat = np.expm1(oof)
    Hit_score = np.sum([1 for i in np.abs((y - yhat) / y)  if i <= 0.1 ])
    Hit_score = round(Hit_score, 4) /len(train) * 10000
    print('Hit rate : {}'.format(Hit_score))
    
    #display_importances(feature_importance_df)
    return predictions, Hit_score, oof


In [57]:
data = Feature_engineer(data)
gc.collect()

266

In [59]:
begin = time.time()
avg_hit_rate = 0
prediction_df = pd.DataFrame()
hit_score_list = []
train_num_list = []
building_df_list   = []
cat_cols = ['city', 'location_2']

train = data[:59989]
test = data[59989:]

# A model every building_type
for b_type in train['building_type'].unique():
    temp_train = train[train['building_type'] == b_type]
    temp_test  = test[test['building_type'] == b_type]
    temp_train.drop(['building_type'], axis =1, inplace =True)
    temp_test.drop(['building_type'], axis =1, inplace =True)
    
    target_df = train.groupby(['city', 'town']).agg({'building_area' : ['mean', 'median'], 'land_area' : ['mean', 'median'], 'total_price' : ['mean', 'median']}).reset_index()
    target_df.columns = [i[0] + '_' + i[1]  if i[1] != '' else i[0] for i in target_df.columns.tolist()]
    target_df['price_land_rate_median'] = np.log1p(target_df['total_price_median']) / target_df['land_area_median']
    target_df['price_building_rate_median'] = np.log1p(target_df['total_price_median']) / target_df['building_area_median']
    target_df['price_land_rate_mean'] = np.log1p(target_df['total_price_mean']) / target_df['land_area_mean']
    target_df['price_building_rate_mean'] = np.log1p(target_df['total_price_mean']) / target_df['building_area_mean']
    
    combine_cols = ['city', 'town', 'price_land_rate_median', 'price_building_rate_median', 'price_land_rate_mean', 'price_building_rate_mean']
    train = pd.merge(train, target_df[combine_cols], on =['city', 'town'], how='left')
    test = pd.merge(test, target_df[combine_cols], on =['city', 'town'], how='left')
    
    preds, hit_score, oof = lgb_model(10, temp_train, temp_test, cat_cols, stratified=False)
    temp = pd.DataFrame({'building_id' : temp_test['building_id'], 'total_price' : preds})
    prediction_df = pd.concat([prediction_df, temp], axis=0)
    
    print('building_type : {}'.format(b_type))
    print('Train_num: {}'.format(len(temp_train)))
    print('Test_num: {}'.format(len(temp_test)))
    print('-'*1000)
    hit_score_list.append(hit_score)
    train_num_list.append(len(temp_train))
    building_df_list.append(b_type)
    avg_hit_rate += hit_score / 60000 * len(temp_train)
    print(avg_hit_rate)
Result_df = pd.DataFrame({'Building_tye' : building_df_list,
                        'Train_num' : train_num_list,
                        'Hit_score' : hit_score_list})

print('Avg hit_score : {}'.format(avg_hit_rate))
print('總共花：{} 分'.format((time.time() - begin) / 60))
print('現在時間 ： {}'.format(datetime.datetime.today().strftime('%m-%d-%H-%M')))
Submission(prediction_df['building_id'], np.expm1(prediction_df['total_price']))

328
Training until validation scores don't improve for 5000 rounds.
[3000]	training's l2: 0.00246514	training's Hit_Score: 9559	valid_1's l2: 0.035271	valid_1's Hit_Score: 5636
[6000]	training's l2: 0.000601547	training's Hit_Score: 9961	valid_1's l2: 0.0346754	valid_1's Hit_Score: 5683
[9000]	training's l2: 0.000253314	training's Hit_Score: 9997	valid_1's l2: 0.0345888	valid_1's Hit_Score: 5759
[12000]	training's l2: 0.000146223	training's Hit_Score: 9999	valid_1's l2: 0.0345808	valid_1's Hit_Score: 5731
Early stopping, best iteration is:
[9125]	training's l2: 0.000246442	training's Hit_Score: 9998	valid_1's l2: 0.0345895	valid_1's Hit_Score: 5787
fold 1 hit_score : 5787.476280834914
------------------------------
Training until validation scores don't improve for 5000 rounds.
[3000]	training's l2: 0.0024867	training's Hit_Score: 9540	valid_1's l2: 0.0277539	valid_1's Hit_Score: 5835
[6000]	training's l2: 0.00060277	training's Hit_Score: 9963	valid_1's l2: 0.027401	valid_1's Hit_Score

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

[18000]	training's l2: 0.000193834	training's Hit_Score: 9997	valid_1's l2: 0.0706438	valid_1's Hit_Score: 4380
[21000]	training's l2: 0.000142611	training's Hit_Score: 9998	valid_1's l2: 0.0706292	valid_1's Hit_Score: 4368
Early stopping, best iteration is:
[18021]	training's l2: 0.000193356	training's Hit_Score: 9997	valid_1's l2: 0.0706441	valid_1's Hit_Score: 4386
fold 7 hit_score : 4386.406544996853
------------------------------
Training until validation scores don't improve for 5000 rounds.
[3000]	training's l2: 0.00945992	training's Hit_Score: 7278	valid_1's l2: 0.0693974	valid_1's Hit_Score: 3958
[6000]	training's l2: 0.00261327	training's Hit_Score: 9432	valid_1's l2: 0.0672537	valid_1's Hit_Score: 4267
[9000]	training's l2: 0.00101106	training's Hit_Score: 9908	valid_1's l2: 0.0667524	valid_1's Hit_Score: 4386
[12000]	training's l2: 0.00049324	training's Hit_Score: 9978	valid_1's l2: 0.0665971	valid_1's Hit_Score: 4399
Early stopping, best iteration is:
[9555]	training's l2:

In [ ]:
#Record
#local hit_score : 5600, public : 5773    Handle target-encoding on city and town
#local hit_score : 5621, public : 5847    Handle target-encoding on city and town + FE
#local hit_score : 5638, public : 5737    Handle target-encoding on city and town + FE + Statistic